In [33]:

import requests
import pandas as pd
import time

df_stations = pd.read_csv("bike_stations.csv")

fsquare_api_key = "fsq3mY8gA3kNr3ohupZu2Q24lP6dQWjWE7gDtfcipnZRMsA="

headers = {
    "Accept": "application/json",
    "Authorization": fsquare_api_key
}

params = {
    "ll": "24.451202,54.334510",  # lat,long
    "radius": 1000,
    "categories": "13065",  # restaurants
    "limit": 10
}

url = "https://api.foursquare.com/v3/places/search"
response = requests.get(url, headers=headers, params=params)
results = response.json()
print(results["results"][0])
#results

{'fsq_id': '5eee3a864f80b40008a49ab4', 'categories': [{'id': 13236, 'name': 'Italian Restaurant', 'short_name': 'Italian', 'plural_name': 'Italian Restaurants', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/italian_', 'suffix': '.png'}}], 'chains': [], 'closed_bucket': 'VeryLikelyOpen', 'distance': 96, 'geocodes': {'main': {'latitude': 24.452015, 'longitude': 54.334343}}, 'link': '/v3/places/5eee3a864f80b40008a49ab4', 'location': {'address': 'Al Bateen', 'country': 'AE', 'cross_street': '', 'formatted_address': 'Al Bateen, أبوظبي', 'locality': 'أبوظبي', 'region': 'أبوظبي'}, 'name': 'Napo Ristorante', 'related_places': {}, 'timezone': 'Asia/Dubai'}


# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [27]:
df_stations = pd.read_csv("bike_stations.csv")
poi_results = []
for index, row in df_stations.iterrows():
    lat = row["latitude"]
    lon = row["longitude"]
    station_name = row["name"]

    params = {
        "ll": f"{lat},{lon}",
        "radius": 1000,
        "categories": "13065",  # Restaurants
        "limit": 10
    }

    url = "https://api.foursquare.com/v3/places/search"

    try:
        response = requests.get(url, headers=headers, params=params)
        results = response.json()

        for place in results.get("results", []):
            
            poi_results.append({
                "station": station_name,
                "station_lat": lat,
                "station_lon": lon,
                "place_name": place["name"],
                "category": place["categories"][0]["name"] if place["categories"] else "Unknown",
                "distance": place.get("distance", None),
                "poi_latitude": place.get("geocodes", {}).get("main", {}).get("latitude"),
                "poi_longitude": place.get("geocodes", {}).get("main", {}).get("longitude")
            })

        time.sleep(0.2)  # Respect rate limit
    except Exception as e:
        print(f"Failed for station {station_name}: {e}")

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [32]:
#poi_results
#results

Put your parsed results into a DataFrame

In [31]:
df_fsq = pd.DataFrame(poi_results)
df_fsq.head()
#df_fsq



,station,station_lat,station_lon,place_name,category,distance,poi_latitude,poi_longitude
0,AUH - Marasy,24.451202,54.33451,Napo Ristorante,Italian Restaurant,96,24.452015,54.334343
1,AUH - Marasy,24.451202,54.33451,The Abu Dhabi EDITION,Restaurant,242,24.451164,54.336928
2,AUH - Marasy,24.451202,54.33451,Market At Edition,Theme Restaurant,202,24.452274,54.336298
3,AUH - Marasy,24.451202,54.33451,Tasha's,African Restaurant,556,24.452782,54.339680
4,AUH - Marasy,24.451202,54.33451,Saddle,Coffee Shop,621,24.453458,54.340019


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating